# SQL grouping and summarizing data (exercises)

## Packages and database files needed for this project

*Note:* This section is repeated in several notebooks in order to make them functional as independent lectures.

Import needed packages:

In [ ]:
import urllib.request    # needed for download of the example database
import shutil            # needed for unziping of the example database

from sqlalchemy import * # package providing unified access to various databases

If necessary, download the database:

In [ ]:
import urllib.request    # needed for download of the example database
import shutil            # needed for unziping of the example database
import os                # for checking existence/removing of a file

In [ ]:
if not os.path.isfile("chinook.db"):
    urllib.request.urlretrieve("https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip", "chinook.zip")
    shutil.unpack_archive("chinook.zip")
    os.remove("chinook.zip")

In this session we use the magic database connector. Install `ipython-sql` when necessary.

In [ ]:
#pip install ipython-sql #In case the SQLite not working in Jupyter
%load_ext sql
%sql sqlite:///chinook.db

## Exercise: summaries per group

### Question

Using SQL only create a table containing summary info of the `tracks` table grouped by albums (so, by the `AlbumId` column).  
In the result table report columns:
- `AlbumId`
- `TracksNum`: the total count of tracks in the album
- `TotalPrice`: sum of all track `UnitPrice`s
- `MeanTrackSec`: based on `Milliseconds`, the average time duration of tracks expressed in seconds

Order the final table with decreasing `TotalPrice`. Show first 10 rows.    
Additionally, you may `ROUND` the prices to two positions and times to full seconds.

### Solution

In [ ]:
%%sql
SELECT 
    AlbumId, 
    COUNT(TrackId) AS TracksNum, 
    ROUND(SUM(UnitPrice),2) AS TotalPrice,
    ROUND(AVG(Milliseconds) / 1000) AS MeanTrackSec
  FROM tracks
  GROUP BY AlbumId
  ORDER BY TotalPrice DESC
  LIMIT 10

## Exercise: summaries per group (with pandas)

Use another strategy to generate the table from the previous exercise.  
Use SQL only to get the complete `tracks` table into a Pandas `DataFrame`.  
Then, perform the same calculations using Pandas `groupby` and `agg` commands.  

Now, imagine that the database is very large and remote, located on multiple servers somewhere in the world:
- What data are transmitted over the database connection in both exercises?
- Where are the calculations performed in both exercises?
- Which approach would scale better?

In [ ]:
import pandas as pd
engine = create_engine("sqlite:///chinook.db")
sql = text("SELECT * FROM tracks")
df = pd.read_sql(sql, engine)
df = df.groupby('AlbumId').agg(
  TracksNum=pd.NamedAgg(column="AlbumId", aggfunc=len),
  TotalPrice=pd.NamedAgg(column="UnitPrice", aggfunc=sum),
  MeanTrackSec=pd.NamedAgg(column="Milliseconds", aggfunc=lambda x: round(sum(x)/len(x)/1000))
)
df = df.sort_values(["TotalPrice"], ascending=False)
df.head(10)

## Exercise: filtering based on summary result

### Question

Find `AlbumId`s that have the number of tracks between 18 and 20. Show 5 rows.

### Soultion

In [ ]:
%%sql
SELECT
    AlbumId,
    COUNT(TrackId) AS TracksNum
  FROM tracks
  GROUP BY AlbumId
  HAVING TracksNum BETWEEN 18 AND 20
  ORDER BY AlbumId
  LIMIT 5

## Exercise: concatenating texts in a group

### Question

Based on `tracks` grouped by `AlbumId`, create a table with two columns:
- `AlbumId`
- `Tracks`: a semicolon-separated concatenated track names of the album

Show some 5 rows.

### Solution

In [ ]:
%%sql
SELECT AlbumId, GROUP_CONCAT(Name,';') AS Tracks
  FROM tracks
  GROUP BY AlbumId
  LIMIT 5